In [1]:
library('rorcid')
library("tidyverse")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
packageVersion("rorcid")         # Check package version of dplyr

[1] ‘0.6.4’

In [3]:
library("utils")
library("tidyverse")
filename <- "data/journal_Annals_of_Glaciology_1.0.0.csv"
df <- read.csv((filename), header=TRUE, sep = "|", stringsAsFactors = FALSE)


In [4]:
# split writers
writers <- strsplit(as.character(df$writer), split = "#", fixed = TRUE)

In [11]:
# cut the writer column to individual writers and return the tokkenised writer name 
namesfilter  <- function (writerlist) {
            # index for list
            i <- 1
            # the list of the result
            filterednames <- list()
            for ( wi in 1:length(writerlist) ) {
               print(writerlist[[wi]])
               writername <- try(strsplit( 
                   strsplit( 
                       writerlist[[wi]], 
                       split = "--", fixed = TRUE )[[1]][1],
                   split = " ", fixed = TRUE )[[1]]
                                 ,silent=TRUE)
               if(class(writername) %in% 'try-error') {print(writerlist[[wi]])}
               # drop empty
               for ( ni in length(writername):1 ) {
                   if ( str_length(writername[[ni]]) < 1 ) {
                       writername <- writername[-ni]
                   } else {
                       writername[ni] <- str_to_lower(writername[[ni]])
                   }
               }
               # add to result
               filterednames[[i]] = writername
               i <- i + 1
            }            
            return(filterednames)
        }

In [13]:
# itterate over df 
for (row in 1:nrow(df)) {
    if ( nchar(toString(df$doi[row] )) > 1 ) {
        Sys.sleep(1)
        # get the orcid data
        orcidwriters <- try(
            orcid_doi(dois=toString(df$doi[row]), fuzzy=FALSE)[[1]]$`orcid-identifier.path`,
            silent=TRUE)
        if(class(orcidwriters) %in% 'try-error') {next}
        if (length(orcidwriters) == 0) {next}
        # get out writers name
        names <- namesfilter(writers[row][[1]])
        # itterate over orcid data
        for ( owi in 1:length(orcidwriters)) {
            # get the name of the orcid id
            res <- orcid_id(orcid = orcidwriters[[owi]])
            family_name <- try( str_to_lower(res[[1]]$name$`family-name`[[1]]),
                               silent=TRUE)
            given_name <- try(str_to_lower(res[[1]]$name$`given-name`[[1]]),
                               silent=TRUE)
            if(class(family_name) %in% 'try-error') {next}
            if(class(given_name) %in% 'try-error') {next}
            # if no family name just skip
            if (identical(family_name, character(0))) {next}

            # find index of the writer for the orcidid
            #  match family names
            thiswirter_index <- 1
            thiswirter <- list()
            for ( wi in 1:length(names) ) {
                # itterate over the writer as the name order can be different
                for ( wii in 1:length(names[[wi]]) ) {
                    # we use str_detect as family name is not always one, example spanish
                    if ( str_detect(family_name, names[[wi]][[wii]] ) ) {
                        thiswirter[thiswirter_index] <- wi
                        thiswirter_index <- thiswirter_index+1
                    }
                }                                
            }
            # if multiple family name match we try to filter by the given name
            if ( length(thiswirter) < 1 ) {
                print(str_c("Error in: ", row, " owi:", orcidwriters[[owi]] ) )
                next
            }
            truewriter <- thiswirter[[1]]
            if ( length(thiswirter) > 1 & !identical(given_name, character(0)) ) {
                # itterate ofer the names where the family name match
                for ( wi in 1:length(thiswirter) )  {
                    userindex <- thiswirter[[wi]]
                    # if we have more data than just the familyname
                    if (length(names[[userindex]]) > 1 ) {
                        # itterate over the data
                        for ( wii in 1:length(names[[userindex]]) ) {
                            # if the given name match
                            if ( str_detect(given_name, names[[userindex]][[wii]] ) ){
                                truewriter <- userindex
                                break
                            }
                        }
                    } 
                }
            }
            # enrich data 
            # if index have no orcid id alread we add            
            if ( !str_detect( writers[row][[1]][truewriter], "---" )) {
                print(str_c("Add: ", orcidwriters[[owi]] ))
                writers[row][[1]][truewriter] <- str_c( writers[row][[1]][truewriter], "---", orcidwriters[[owi]] )                 
            }
        }
        # make the enriched writerstring
        result <- ""
        for ( wi in 1:length(writers[row][[1]]) ) {
            if ( wi < length(writers[row][[1]]) ) {
                result <- str_c(result, writers[row][[1]][wi], "#" )
            } else {
                result <- str_c(result, writers[row][[1]][wi] )
            }
        }
        # update df
        df$writer[row] <- result        
    };
}

[1] "Jon Ove Hagen--Department of Physical Geography, University of Oslo"
[1] "Bernd Etzelmüller--Department of Physical Geography, University of Oslo"
[1] "Anne-Marie Nuttall--Bristol Glaciology Centre, School of Geographical Sciences, University of Bristol---0000-0002-6716-3491"
[1] "Roberto Udisti--Department of Chemistry, University of Florence"
[1] "Silvia Becagli--Department of Chemistry, University of Florence"
[1] "Silvia Benassai--Department of Chemistry, University of Florence"
[1] "Martine De Angelis"
[1] "Margareta E. Hansson--Department of Physical Geography and Quaternary Geology, Stockholm University"
[1] "Jean Jouzel"
[1] "Jacob Schwander--Physics Institute, University of Bern"
[1] "Jørgen P. Steffensen--Department of Geophysics, The Niels Bohr Institute, University of Copenhagen"
[1] "Rita Traversi--Department of Chemistry, University of Florence"
[1] "Eric Wolff--British Antarctic Survey, Natural Environment Research Council---0000-0002-5914-8531"


Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”


[1] "Akira Hori--Institute of Low Temperature Science, Hokkaido University"
[1] "Takeo Hondoh--Institute of Low Temperature Science, Hokkaido University---0000-0003-4129-022X"
[1] "Mitsugu Oguro"
[1] "Vladimir Ya. Lipenkov--Arctic and Antarctic Research Institute"


Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”


[1] "Douglas R. MacAyeal--Department of Geophysical Sciences---0000-0003-0647-6176"
[1] "Dorian S. Abbot--Department of Geophysical Sciences"
[1] "Olga V. Sergienko--Atmospheric and Oceanic Sciences Program---0000-0002-5764-8815"
[1] "T. M. Jordan--University of Bristol---0000-0002-2096-8858"
[1] "D. Z. Besson--Kansas University---0000-0001-6733-963X"
[1] "I. Kravchenko--University of Nebraska"
[1] "U. Latif--Kansas University"
[1] "B. Madison--Kansas University"
[1] "A. Nokikov--Kansas University"
[1] "A. Shultz--Kansas University"


...

Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”

[1] "Peter D. Ditlevsen--The Niels Bohr Institute"
[1] "Susanne Ditlevsen--Department of Biostatistics"
[1] "Katrine K. Andersen--The Niels Bohr Institute"
[1] "Add: 0000-0003-4178-2195"
[1] "Add: 0000-0002-1998-2783"


Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”


[1] "Surendra Adhikari"
[1] "Philippe Huybrechts"
[1] "Add: 0000-0003-1021-6860"
[1] "Add: 0000-0003-1406-0525"


Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”
Warning message:
“Unknown or uninitialised column: `orcid-identifier.path`.”


In [9]:
# save df as csv
write.table(df, file = str_replace(filename, "journal_", "journal_enrichment_"), row.names = FALSE, sep="|")